In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from companies.models import Company
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [3]:
from django.db.models import Count, F, Value
from django.db.models.functions import Length, Upper
from django.db.models.lookups import GreaterThan

In [6]:
com1 = Company.objects.filter(num_employees__gt=F("num_chairs"))
print(com1)

<QuerySet [<Company: Company BBB>, <Company: Company CCC>]>


In [7]:
print(com1.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs" FROM "companies_company" WHERE "companies_company"."num_employees" > ("companies_company"."num_chairs")


In [10]:
Company.objects.filter(num_employees__gt=F("num_chairs") + F("num_tables"))

<QuerySet [<Company: Company CCC>]>

In [14]:
Company.objects.filter(num_employees__gt=F("num_chairs")*2)

<QuerySet [<Company: Company CCC>]>

## Annotation

In [23]:
company = (Company.objects.filter(num_employees__gt=F("num_chairs")
            ).annotate(chairs_needed=F("num_employees")-F("num_chairs"))).filter(chairs_needed__gt=50)
# print(company)
for c in company:
    print(f"{c.name} num_employees={c.chairs_needed} num_chairs={c.num_chairs} chairs_needed={c.chairs_needed}")

Company CCC num_employees=60 num_chairs=40 chairs_needed=60


In [16]:
print(company.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs", ("companies_company"."num_employees" - "companies_company"."num_chairs") AS "chairs_needed" FROM "companies_company" WHERE ("companies_company"."num_employees" > ("companies_company"."num_chairs") AND ("companies_company"."num_employees" - "companies_company"."num_chairs") > 50)


In [20]:
company[0].num_employees

100

In [16]:
company.num_chairs

30

In [18]:
company.chairs_needed

20

In [29]:
company = Company.objects.create(name="Google", ticker=Upper(Value("goog")))
company.refresh_from_db()

In [30]:
company.ticker

'GOOG'

In [24]:
print(company.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs", ("companies_company"."num_employees" - "companies_company"."num_chairs") AS "chairs_needed" FROM "companies_company" WHERE ("companies_company"."num_employees" > ("companies_company"."num_chairs") AND ("companies_company"."num_employees" - "companies_company"."num_chairs") > 50)


## Orderby, Lenght

In [34]:
Company.objects.order_by("-name")

<QuerySet [<Company: Google>, <Company: Company CCC>, <Company: Company BBB>, <Company: Company AAA>]>

In [31]:
Company.objects.order_by(Length("name").asc())

<QuerySet [<Company: Google>, <Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>]>

In [32]:
Company.objects.order_by(Length("name").desc())

<QuerySet [<Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>, <Company: Google>]>

In [39]:
Company.objects.filter(num_employees__gt=F("num_chairs"))

<QuerySet [<Company: Company BBB>, <Company: Company CCC>]>

In [38]:
Company.objects.filter(GreaterThan(F("num_employees"), F("num_chairs")))

<QuerySet [<Company: Company BBB>, <Company: Company CCC>]>

## Greater than return True False

In [43]:
company = Company.objects.annotate(need_chairs=GreaterThan(F("num_employees"), F("num_chairs")))
for c in company:
    print(f"{c.name} num_employees={c.num_employees} num_chairs={c.num_chairs} chairs_needed={c.need_chairs}")

Company AAA num_employees=120 num_chairs=150 chairs_needed=False
Company BBB num_employees=50 num_chairs=30 chairs_needed=True
Company CCC num_employees=100 num_chairs=40 chairs_needed=True
Google num_employees=0 num_chairs=0 chairs_needed=False
